In [2]:
# 检查GPU环境。如提示无GPU，需要点击「修改」-「笔记本设置」-「GPU-T4」，然后点击右上角的连接按键。
!nvidia-smi

Wed Aug  2 10:30:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# 导入需要用到的Python工具包
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture
from google.colab import drive

In [4]:
# 设置项目需要的各种路径，比如基础模型存放路径、LoRA模型训练后的存放路径等
root_dir = os.path.abspath("/content")
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")
training_dir = os.path.join(root_dir, "LoRA")
pretrained_model = os.path.join(root_dir, "pretrained_model")
vae_dir = os.path.join(root_dir, "vae")
config_dir = os.path.join(training_dir, "config")

accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir, "tools")
finetune_dir = os.path.join(repo_dir, "finetune")

# TODO：在新的notebook中，这段代码可以删除
for store in [
    "root_dir",
    "deps_dir",
    "repo_dir",
    "training_dir",
    "pretrained_model",
    "vae_dir",
    "accelerate_config",
    "tools_dir",
    "finetune_dir",
    "config_dir",
]:
    with capture.capture_output() as cap:
        del cap

安装基础环境

In [5]:
repo_url = "https://github.com/Linaqruf/kohya-trainer"
bitsandytes_main_py = "/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py"
branch = ""
install_xformers = True
verbose = False

def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents


def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)


def clone_repo(url):
    if not os.path.exists(repo_dir):
        os.chdir(root_dir)
        !git clone {url} {repo_dir}
    else:
        os.chdir(repo_dir)
        !git pull origin {branch} if branch else !git pull


def install_dependencies():
    s = getoutput('nvidia-smi')

    !pip install {'-q' if not verbose else ''} --upgrade -r requirements.txt
    !pip install {'-q' if not verbose else ''} torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U

    if install_xformers:
        !pip install {'-q' if not verbose else ''} xformers==0.0.19 triton==2.0.0 -U

    from accelerate.utils import write_basic_config

    if not os.path.exists(accelerate_config):
        write_basic_config(save_location=accelerate_config)


def main():
    os.chdir(root_dir)

    for dir in [
        deps_dir,
        training_dir,
        config_dir,
        pretrained_model,
        vae_dir
    ]:
        os.makedirs(dir, exist_ok=True)

    clone_repo(repo_url)

    os.chdir(repo_dir)
    !apt --fix-broken install
    !apt install aria2 {'-qq' if not verbose else ''}

    install_dependencies()
    time.sleep(3)

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["BITSANDBYTES_NOWELCOME"] = "1"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"

    cuda_path = "/usr/local/cuda-11.8/targets/x86_64-linux/lib/"
    ld_library_path = os.environ.get("LD_LIBRARY_PATH", "")
    os.environ["LD_LIBRARY_PATH"] = f"{ld_library_path}:{cuda_path}"

main()


Cloning into '/content/kohya-trainer'...
remote: Enumerating objects: 2481, done.
remote: Counting objects: 100% (1149/1149), done.
remote: Compressing objects: 100% (369/369), done.
remote: Total 2481 (delta 887), reused 919 (delta 780), pack-reused 1332
Receiving objects: 100% (2481/2481), 4.91 MiB | 852.00 KiB/s, done.
Resolving deltas: 100% (1642/1642), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120500 files and directories currently installed.)
Preparing to unpack .

下载基础模型

In [6]:
# 下面提供了一些模型的下载链接，你可以到Hugging Face和CivitAI上获取更多模型。

# "Anything-v3-1": "https://huggingface.co/cag/anything-v3-1/resolve/main/anything-v3-1.safetensors",
# "AnyLoRA": "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/AnyLoRA_noVae_fp16-pruned.safetensors",
# "AnyLoRA-anime-mix": "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AAM_Anylora_AnimeMix.safetensors",
# "AnimePastelDream": "https://huggingface.co/Lykon/AnimePastelDream/resolve/main/AnimePastelDream_Soft_noVae_fp16.safetensors",
# "Chillout-mix": "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/chillout_mix-pruned.safetensors",
# "OpenJourney-v4": "https://huggingface.co/prompthero/openjourney-v4/resolve/main/openjourney-v4.ckpt",
# "Stable-Diffusion-v1-5": "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/stable_diffusion_1_5-pruned.safetensors"

In [7]:
# 下载你要用的基础模型。这些我用的是AnyLoRA基础模型，你可以根据喜欢进行更换。
# 真人风格推荐用Chillout-mix 这类模型
# 卡通风格推荐AnyLoRA、Anything这类模型
# pretrained_model_name_or_path = "/content/pretrained_model/AnyLoRA_noVae_fp16-pruned.safetensors"
# !wget -c https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/AnyLoRA_noVae_fp16-pruned.safetensors -O $pretrained_model_name_or_path

pretrained_model_name_or_path = "/content/pretrained_model/moyou.safetensors"
!wget -c https://huggingface.co/wind1/MoYou/resolve/main/%E9%99%90%E6%97%B6%E7%8B%AC%E5%8D%A0%EF%B8%B1%E5%A2%A8%E5%B9%BD%E4%BA%BA%E9%80%A0%E4%BA%BA_v1010%E5%AE%8C%E6%95%B4.safetensors -O $pretrained_model_name_or_path

--2023-08-02 10:34:07--  https://huggingface.co/wind1/MoYou/resolve/main/%E9%99%90%E6%97%B6%E7%8B%AC%E5%8D%A0%EF%B8%B1%E5%A2%A8%E5%B9%BD%E4%BA%BA%E9%80%A0%E4%BA%BA_v1010%E5%AE%8C%E6%95%B4.safetensors
Resolving huggingface.co (huggingface.co)... 52.84.162.49, 52.84.162.101, 52.84.162.22, ...
Connecting to huggingface.co (huggingface.co)|52.84.162.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c8/3c/c83ce79715db76543fa20d1d6c44c50e979b06f456884dab974eccc49543ad99/6a226dd292a983b3ed4987402453ad4d954b77825c1cf99d39d8746909791761?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27%25E9%2599%2590%25E6%2597%25B6%25E7%258B%25AC%25E5%258D%25A0%25EF%25B8%25B1%25E5%25A2%25A8%25E5%25B9%25BD%25E4%25BA%25BA%25E9%2580%25A0%25E4%25BA%25BA_v1010%25E5%25AE%258C%25E6%2595%25B4.safetensors%3B+filename%3D%22%C3%A9%C2%99%C2%90%C3%A6%C2%97%C2%B6%C3%A7%C2%8B%C2%AC%C3%A5%C2%8D%C2%A0%C3%AF%C2%B8%C2%B1%C3%A5%C2%A2%C2%A8%C3%A5%

准备数据

In [8]:
train_data_dir = os.path.join(root_dir, "LoRA/train_data/hb_cartoon")
os.makedirs(train_data_dir, exist_ok=True)

print(f"Your train data directory : {train_data_dir}")

Your train data directory : /content/LoRA/train_data/hb_cartoon


In [9]:
# 如果使用提前准备的赫本图片，可以执行下面这段代码
# 如果你想使用自己的图片训练LoRA，可以将你的图片上传到/content/LoRA/train_data/hb_cartoon 路径下，跳过这段代码。

import os
if not os.path.isdir("ai_painting_journey"):
  !git clone https://github.com/NightWalker888/ai_painting_journey.git

if not os.listdir("/content/LoRA/train_data/hb_cartoon"):
  !tar -xvf ai_painting_journey/live/herburn_images.tar -C /content/LoRA/train_data/

train_data_dir = os.path.join(root_dir, "LoRA/train_data/herburn_images")

Cloning into 'ai_painting_journey'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 8), reused 28 (delta 5), pack-reused 0
Receiving objects: 100% (33/33), 10.56 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (8/8), done.
herburn_images/
herburn_images/._herben_cartoon16.png
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
herburn_images/herben_cartoon16.png
herburn_images/._heben1.jpg
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemWhereFroms'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
herburn_images/heben1.jpg
herburn_images/._heben3.jpg
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.x

# **操作说明: 需要将你准备的照片放置在上面的路径下，比如这里的路径是：/content/LoRA/train_data/hb_cartoon。直接将图片拖到文件夹中就行！确保文件上传完成后，到下一步。10图以上，多少随意。**

In [10]:
# 使用BLIP模型给你的图片加上prompt，用于训练。
# BLIP是一个多模态生成算法，输入图片，得到图片的prompt描述信息。

import os

os.chdir(finetune_dir)

batch_size = 8
max_data_loader_n_workers = 2
beam_search = True
min_length = 5
max_length = 75
recursive = False
verbose_logging = True

config = {
    "_train_data_dir" : train_data_dir,
    "batch_size" : batch_size,
    "beam_search" : beam_search,
    "min_length" : min_length,
    "max_length" : max_length,
    "debug" : verbose_logging,
    "caption_extension" : ".caption",
    "max_data_loader_n_workers" : max_data_loader_n_workers,
    "recursive" : recursive
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python make_captions.py {args}"

os.chdir(finetune_dir)
!{final_args}

load images from /content/LoRA/train_data/herburn_images
found 20 images.
loading BLIP caption: https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
100% 1.66G/1.66G [00:07<00:00, 240MB/s]
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
BLIP loaded
  0% 0/3 [00:00<?, ?it/s]Could not load image path / 画像を読み込めません: /content/LoRA/train_data/herburn_images/._herben_cartoon4.png, error: cannot identify image file '/content/LoRA/train_data/herburn_images/._herben_cartoon4.png'
Could not load image path / 画像を読み込めません: /content/LoRA/train_data/herburn_images/._heben1.jpg, error: cannot identify image file '/content/LoRA/train_data/herburn_images/._heben1.jpg'
Could not load image path / 画像を読み込めません: /content/LoRA/train_data/herburn_images/._heben19.jpg, error: cannot identify image file '/content/LoRA/train_data/herburn_images/._heben19.jpg'
Could not load image path / 画像を読み込めません: /cont

训练配置

In [11]:
project_name = "hb_pro"
vae = "" # 使用基础模型中的VAE
output_dir = os.path.join(root_dir, "LoRA/output/hb_pro")

sample_dir = os.path.join(output_dir, "sample")
for dir in [output_dir, sample_dir]:
    os.makedirs(dir, exist_ok=True)

print("Project Name: ", project_name)
print(
    "Pretrained Model Path: ", pretrained_model_name_or_path
) if pretrained_model_name_or_path else print("No Pretrained Model path specified.")

print("VAE Path: ", vae) if vae else print("No VAE path specified.")
print("Output Path: ", output_dir)

Project Name:  hb_pro
Pretrained Model Path:  /content/pretrained_model/moyou.safetensors
No VAE path specified.
Output Path:  /content/LoRA/output/hb_pro


In [12]:
# 这段代码用于预处理数据，将我们的训练数据、正则化数据处理成训练模型能用的dataloader
# 数据增广是非常关键的操作，比如图像翻转、图像颜色扰动、图像朝着目标风格的迁移等。这些都是后期可调的。

import os
import toml
import glob

dataset_repeats = 20
activation_word = ""
caption_extension = ".caption"
resolution = 512
flip_aug = True
keep_tokens = 0

def parse_folder_name(folder_name, default_num_repeats, default_class_token):
    folder_name_parts = folder_name.split("_")

    if len(folder_name_parts) == 2:
        if folder_name_parts[0].isdigit():
            num_repeats = int(folder_name_parts[0])
            class_token = folder_name_parts[1].replace("_", " ")
        else:
            num_repeats = default_num_repeats
            class_token = default_class_token
    else:
        num_repeats = default_num_repeats
        class_token = default_class_token

    return num_repeats, class_token

def find_image_files(path):
    supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp")
    return [file for file in glob.glob(path + '/**/*', recursive=True) if file.lower().endswith(supported_extensions)]

def process_data_dir(data_dir, default_num_repeats, default_class_token, is_reg=False):
    subsets = []

    images = find_image_files(data_dir)
    if images:
        subsets.append({
            "image_dir": data_dir,
            "class_tokens": default_class_token,
            "num_repeats": default_num_repeats,
            **({"is_reg": is_reg} if is_reg else {}),
        })

    for root, dirs, files in os.walk(data_dir):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            images = find_image_files(folder_path)

            if images:
                num_repeats, class_token = parse_folder_name(folder, default_num_repeats, default_class_token)

                subset = {
                    "image_dir": folder_path,
                    "class_tokens": class_token,
                    "num_repeats": num_repeats,
                }

                if is_reg:
                    subset["is_reg"] = True

                subsets.append(subset)

    return subsets


train_subsets = process_data_dir(train_data_dir, dataset_repeats, activation_word)
print(train_subsets)
# reg_subsets = process_data_dir(reg_data_dir, dataset_repeats, activation_word, is_reg=True)

# subsets = train_subsets + reg_subsets
subsets = train_subsets

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": caption_extension,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": resolution,
            "min_bucket_reso": 320 if resolution > 640 else 256,
            "max_bucket_reso": 1280 if resolution > 640 else 1024,
            "caption_dropout_rate": 0,
            "caption_tag_dropout_rate": 0,
            "caption_dropout_every_n_epochs": 0,
            "flip_aug": flip_aug,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": subsets,
        }
    ],
}

dataset_config = os.path.join(config_dir, "dataset_config.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(dataset_config, "w") as f:
    f.write(config_str)

print(config_str)

[{'image_dir': '/content/LoRA/train_data/herburn_images', 'class_tokens': '', 'num_repeats': 20}]
[[datasets]]
resolution = 512
min_bucket_reso = 256
max_bucket_reso = 1024
caption_dropout_rate = 0
caption_tag_dropout_rate = 0
caption_dropout_every_n_epochs = 0
flip_aug = true
color_aug = false
[[datasets.subsets]]
image_dir = "/content/LoRA/train_data/herburn_images"
class_tokens = ""
num_repeats = 20


[general]
enable_bucket = true
caption_extension = ".caption"
shuffle_caption = true
keep_tokens = 0
bucket_reso_steps = 64
bucket_no_upscale = false



In [13]:
# 可以提供预训练的LoRA模型
# 分别设置text_encoder和UNet的学习率

network_category = "LoRA"

conv_dim = 32
conv_alpha = 16
network_dim = 32
network_alpha = 16
network_weight = ""
network_module = "networks.lora"
network_args = ""

min_snr_gamma = -1
optimizer_type = "AdamW8bit"
optimizer_args = ""
train_unet = True
unet_lr = 1e-4
train_text_encoder = True
text_encoder_lr = 5e-5
lr_scheduler = "constant"
lr_warmup_steps = 0
lr_scheduler_num_cycles = 0
lr_scheduler_power = 0

print("- LoRA Config:")
print(f"  - Min-SNR Weighting: {min_snr_gamma}") if not min_snr_gamma == -1 else ""
print(f"  - Loading network module: {network_module}")
print(f"  - {network_module} linear_dim set to: {network_dim}")
print(f"  - {network_module} linear_alpha set to: {network_alpha}")

if not network_weight:
    print("  - No LoRA weight loaded.")
else:
    if os.path.exists(network_weight):
        print(f"  - Loading LoRA weight: {network_weight}")
    else:
        print(f"  - {network_weight} does not exist.")
        network_weight = ""

print("- Optimizer Config:")
print(f"  - Additional network category: {network_category}")
print(f"  - Using {optimizer_type} as Optimizer")
if optimizer_args:
    print(f"  - Optimizer Args: {optimizer_args}")
if train_unet and train_text_encoder:
    print("  - Train UNet and Text Encoder")
    print(f"    - UNet learning rate: {unet_lr}")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
if train_unet and not train_text_encoder:
    print("  - Train UNet only")
    print(f"    - UNet learning rate: {unet_lr}")
if train_text_encoder and not train_unet:
    print("  - Train Text Encoder only")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
print(f"  - Learning rate warmup steps: {lr_warmup_steps}")
print(f"  - Learning rate Scheduler: {lr_scheduler}")
if lr_scheduler == "cosine_with_restarts":
    print(f"  - lr_scheduler_num_cycles: {lr_scheduler_num_cycles}")
elif lr_scheduler == "polynomial":
    print(f"  - lr_scheduler_power: {lr_scheduler_power}")

- LoRA Config:
  - Loading network module: networks.lora
  - networks.lora linear_dim set to: 32
  - networks.lora linear_alpha set to: 16
  - No LoRA weight loaded.
- Optimizer Config:
  - Additional network category: LoRA
  - Using AdamW8bit as Optimizer
  - Train UNet and Text Encoder
    - UNet learning rate: 0.0001
    - Text encoder learning rate: 5e-05
  - Learning rate warmup steps: 0
  - Learning rate Scheduler: constant


In [14]:
# 设置存储模型的格式，我们为了适配WebUI，使用safetensors格式
# 设置测试用prompt
# 设置训练参数


import toml
import os

lowram = True
enable_sample_prompt = True
sampler = "ddim"  #["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
noise_offset = 0.0
num_epochs = 10
vae_batch_size = 4
train_batch_size = 6
mixed_precision = "fp16"  # ["no","fp16","bf16"]
save_precision = "fp16"  #  ["float", "fp16", "bf16"]
save_n_epochs_type = "save_every_n_epochs"
save_n_epochs_type_value = 1
save_model_as = "safetensors"  # ["ckpt", "pt", "safetensors"]
max_token_length = 225
clip_skip = 2
gradient_checkpointing = False
gradient_accumulation_steps = 1
seed = -1
logging_dir = os.path.join(root_dir, "LoRA/logs")
prior_loss_weight = 1.0
os.chdir(repo_dir)

# sample_str = f"""
#   masterpiece, best quality, a woman with a very short haircut and a pink shirt, looking at viewer, simple background \
#   --n lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry \
#   --w 512 \
#   --h 512 \
#   --l 7 \
#   --s 28
# """

sample_str = f"""
  masterpiece, best quality, 1girl,moyou，very short haircut and a pink shirt, looking at viewer, simple background \
  --n EasyNegativeV2,(badhandv4:1.2), \
  --w 512 \
  --h 512 \
  --l 7 \
  --s 28
"""

config = {
    "model_arguments": {
        "v2": False,
        "v_parameterization": False,
        "pretrained_model_name_or_path": pretrained_model_name_or_path,
        "vae": vae,
    },
    "additional_network_arguments": {
        "no_metadata": False,
        "unet_lr": float(unet_lr) if train_unet else None,
        "text_encoder_lr": float(text_encoder_lr) if train_text_encoder else None,
        "network_weights": network_weight,
        "network_module": network_module,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_args": network_args,
        "network_train_unet_only": True if train_unet and not train_text_encoder else False,
        "network_train_text_encoder_only": True if train_text_encoder and not train_unet else False,
        "training_comment": None,
    },
    "optimizer_arguments": {
        "min_snr_gamma": min_snr_gamma if not min_snr_gamma == -1 else None,
        "optimizer_type": optimizer_type,
        "learning_rate": unet_lr,
        "max_grad_norm": 1.0,
        "optimizer_args": eval(optimizer_args) if optimizer_args else None,
        "lr_scheduler": lr_scheduler,
        "lr_warmup_steps": lr_warmup_steps,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
    },
    "dataset_arguments": {
        "cache_latents": True,
        "debug_dataset": False,
        "vae_batch_size": vae_batch_size,
    },
    "training_arguments": {
        "output_dir": output_dir,
        "output_name": project_name,
        "save_precision": save_precision,
        "save_every_n_epochs": save_n_epochs_type_value if save_n_epochs_type == "save_every_n_epochs" else None,
        "save_n_epoch_ratio": save_n_epochs_type_value if save_n_epochs_type == "save_n_epoch_ratio" else None,
        "save_last_n_epochs": None,
        "save_state": None,
        "save_last_n_epochs_state": None,
        "resume": None,
        "train_batch_size": train_batch_size,
        "max_token_length": 225,
        "mem_eff_attn": False,
        "xformers": True,
        "max_train_epochs": num_epochs,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "seed": seed if seed > 0 else None,
        "gradient_checkpointing": gradient_checkpointing,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "mixed_precision": mixed_precision,
        "clip_skip": clip_skip,
        "logging_dir": logging_dir,
        "log_prefix": project_name,
        "noise_offset": noise_offset if noise_offset > 0 else None,
        "lowram": lowram,
    },
    "sample_prompt_arguments": {
        "sample_every_n_steps": None,
        "sample_every_n_epochs": 1 if enable_sample_prompt else 999999,
        "sample_sampler": sampler,
    },
    "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
    },
    "saving_arguments": {
        "save_model_as": save_model_as
    },
}

config_path = os.path.join(config_dir, "config_file.toml")
prompt_path = os.path.join(config_dir, "sample_prompt.txt")


for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

write_file(config_path, config_str)
write_file(prompt_path, sample_str)

print(config_str)

[model_arguments]
v2 = false
v_parameterization = false
pretrained_model_name_or_path = "/content/pretrained_model/moyou.safetensors"

[additional_network_arguments]
no_metadata = false
unet_lr = 0.0001
text_encoder_lr = 5e-5
network_module = "networks.lora"
network_dim = 32
network_alpha = 16
network_train_unet_only = false
network_train_text_encoder_only = false

[optimizer_arguments]
optimizer_type = "AdamW8bit"
learning_rate = 0.0001
max_grad_norm = 1.0
lr_scheduler = "constant"
lr_warmup_steps = 0

[dataset_arguments]
cache_latents = true
debug_dataset = false
vae_batch_size = 4

[training_arguments]
output_dir = "/content/LoRA/output/hb_pro"
output_name = "hb_pro"
save_precision = "fp16"
save_every_n_epochs = 1
train_batch_size = 6
max_token_length = 225
mem_eff_attn = false
xformers = true
max_train_epochs = 10
max_data_loader_n_workers = 8
persistent_data_loader_workers = true
gradient_checkpointing = false
gradient_accumulation_steps = 1
mixed_precision = "fp16"
clip_skip = 2


训练起来！过程中你可以在LoRA/output/hb_pro/sample目录下看训练过程中的图像生成效果。用于选择自己满意的LoRA模型。

In [15]:
sample_prompt = os.path.join(config_dir, "sample_prompt.txt")
config_file = os.path.join(config_dir, "config_file.toml")
dataset_config = os.path.join(config_dir, "dataset_config.toml")
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts" : sample_prompt,
    "dataset_config" : dataset_config,
    "config_file" : config_file
}

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

accelerate_args = train(accelerate_conf)
train_args = train(train_conf)
final_args = f"accelerate launch {accelerate_args} train_network.py {train_args}"

os.chdir(repo_dir)
!{final_args}

Loading settings from /content/LoRA/config/config_file.toml...
/content/LoRA/config/config_file
prepare tokenizer
update token length: 225
Load dataset config from /content/LoRA/config/dataset_config.toml
prepare images.
found directory /content/LoRA/train_data/herburn_images contains 10 image files
200 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 6
  resolution: (512, 512)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  bucket_no_upscale: False

  [Subset 0 of Dataset 0]
    image_dir: "/content/LoRA/train_data/herburn_images"
    image_count: 10
    num_repeats: 20
    shuffle_caption: True
    keep_tokens: 0
    caption_dropout_rate: 0
    caption_dropout_every_n_epoches: 0
    caption_tag_dropout_rate: 0
    color_aug: False
    flip_aug: True
    face_crop_aug_range: None
    random_crop: False
    token_warmup_min: 1,
    token_warmup_step: 0,
    is_reg: False
    c

LoRA/output/hb_pro/目录下的safetensor文件就是我们得到的LoRA模型，下载到WebUI就可以直接使用。

试试推理？

In [16]:
import os

network_weight = "/content/LoRA/output/hb_pro/hb_pro.safetensors"
network_mul = 1
network_module = "networks.lora"
network_args = ""

v2 = False
v_parameterization = False
prompt = "masterpiece, best quality, 1girl moyou ( ink sketch) fantasy, surreal muted color ( Russ Mills Anna Dittmann)"   # 你要测试的prompt
negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
model = pretrained_model_name_or_path
vae = ""
outdir = "/content/tmp"  # 图片存储的路径
scale = 7
sampler = "euler_a"
steps = 20
precision = "fp16"
width = 512
height = 768
images_per_prompt = 4
batch_size = 4
clip_skip = 2
seed = 1024

final_prompt = f"{prompt} --n {negative}"

config = {
    "v2": v2,
    "v_parameterization": v_parameterization,
    "network_module": network_module,
    "network_weight": network_weight,
    "network_mul": float(network_mul),
    "network_args": eval(network_args) if network_args else None,
    "ckpt": model,
    "outdir": outdir,
    "xformers": True,
    "vae": vae if vae else None,
    "fp16": True,
    "W": width,
    "H": height,
    "seed": seed if seed > 0 else None,
    "scale": scale,
    "sampler": sampler,
    "steps": steps,
    "max_embeddings_multiples": 3,
    "batch_size": batch_size,
    "images_per_prompt": images_per_prompt,
    "clip_skip": clip_skip if not v2 else None,
    "prompt": final_prompt,
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

os.chdir(repo_dir)
!{final_args}

load StableDiffusion checkpoint
loading u-net: <All keys matched successfully>
loading vae: <All keys matched successfully>
loading text encoder: <All keys matched successfully>
Replace CrossAttention.forward to use NAI style Hypernetwork and xformers
loading tokenizer
prepare tokenizer
import network module: networks.lora
load network weights from: /content/LoRA/output/hb_pro/hb_pro.safetensors
create LoRA network from weights
create LoRA for Text Encoder: 72 modules.
create LoRA for U-Net: 192 modules.
enable LoRA for text encoder
enable LoRA for U-Net
weights are loaded: <All keys matched successfully>
pipeline is ready.
iteration 1/1
prompt 1/1: masterpiece, best quality, 1girl moyou ( ink sketch) fantasy, surreal muted color ( Russ Mills Anna Dittmann)
negative prompt: lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry
100% 20/20 [00:23<

点开这个路径：/content/tmp，查看你生成的图片！可以更换代码里的prompt反复尝试几次。权重和图片记得保存本地，不然colab断开就都没了。

在你的WebUI里面玩起来？
- 配合LoRA风格模型
- 配合不同基础模型
- 配合超分模块
- 配合不同的prompt